In [1]:
import pandas as pd
import numpy as np

train_data = pd.read_excel("Data/tr.xlsx",)

In [2]:
over_missing_sample = []
row_value_count = train_data.apply(pd.Series.value_counts,axis=1,dropna=False)

In [3]:
import tools
#Data transform
train_y = pd.DataFrame(train_data['outcome']) 
train_X = pd.DataFrame(train_data.drop(['outcome'],axis=1))

train_X = tools.data_transform(train_X)

In [4]:
threshhold =len(train_X.columns)*0.25
over_missing = row_value_count[np.nan]<=threshhold
class_0 = train_y['outcome']==1

print(sorted(train_y.value_counts()))
train_X = train_X[over_missing|class_0]
train_y = train_y[over_missing|class_0]
print(sorted(train_y.value_counts()))


[879, 51280]
[879, 27076]


In [5]:
# Data split
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(
    train_X ,
    train_y,
    test_size=0.2,
    random_state=42)

print(sorted(train_y.value_counts()))
print(sorted(val_y.value_counts()))

[718, 21646]
[161, 5430]


In [6]:
from imblearn.under_sampling import RandomUnderSampler as RUS

print(sorted(train_y.value_counts()))

rus = RUS(sampling_strategy=0.1,random_state=42)
train_X,train_y = rus.fit_resample(train_X,train_y)

print(sorted(train_y.value_counts()))

[718, 21646]
[718, 7180]


In [7]:
from sklearn.feature_selection import VarianceThreshold as VT

vt= VT(0.2)

vt.fit(train_X)
print(train_X.shape)

train_X = pd.DataFrame(vt.transform(train_X)) 
val_X =  pd.DataFrame(vt.transform(val_X))

print(train_X.shape)

(7898, 66)
(7898, 30)


In [8]:
# Missing value imputation
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import tools
feature_kind = tools.init_feature_kind(train_X)
cont,cate = tools.get_feature_kind(train_X,feature_kind)  

strategy = 'median'

imp_mean = IterativeImputer(max_iter=100,random_state=0)
imp_mean.fit(train_X[cont])

train_X[cont] = imp_mean.transform(train_X[cont])
val_X[cont] = imp_mean.transform(val_X[cont])

print("filled continuous missing value with "+strategy)

strategy = 'constant'

imp = SimpleImputer(missing_values=np.nan, strategy=strategy,fill_value=10.0)
imp.fit(train_X[cate])

train_X[cate] = imp.transform(train_X[cate])
val_X[cate] = imp.transform(val_X[cate])


print("filled nominal missing value with ",strategy)


filled continuous missing value with median
filled nominal missing value with  constant


In [9]:
from sklearn import preprocessing


cont,cate = tools.get_feature_kind(train_X,feature_kind)

scaler = preprocessing.StandardScaler().fit(train_X[cont])
train_X[cont] = scaler.transform(train_X[cont])
val_X[cont] = scaler.transform(val_X[cont])

In [10]:
'''import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import RidgeCV

ridge = RidgeCV(alphas=np.logspace(-6, 6, num=5)).fit(train_X, train_y)
importance = np.abs(ridge.coef_)
feature_names = np.array(train_X.columns)
plt.bar(height=importance[0], x=feature_names)
plt.title("Feature importances via coefficients")
plt.show()


train_X=train_X[feature_names[importance[0]>0.004]]
val_X = val_X[feature_names[importance[0]>0.004]]
print(train_X.shape)'''

'import matplotlib.pyplot as plt\nimport numpy as np\nfrom sklearn.linear_model import RidgeCV\n\nridge = RidgeCV(alphas=np.logspace(-6, 6, num=5)).fit(train_X, train_y)\nimportance = np.abs(ridge.coef_)\nfeature_names = np.array(train_X.columns)\nplt.bar(height=importance[0], x=feature_names)\nplt.title("Feature importances via coefficients")\nplt.show()\n\n\ntrain_X=train_X[feature_names[importance[0]>0.004]]\nval_X = val_X[feature_names[importance[0]>0.004]]\nprint(train_X.shape)'

In [11]:
def set_pandas_display_options() -> None:
    display = pd.options.display
    display.max_columns = 100
    display.max_rows = 100
    display.max_colwidth = 199
    display.width = None
set_pandas_display_options()

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(class_weight={1:1,0:0.1},max_iter=10000) 

parameters = {
    'penalty':('l1', 'l2','elasticnet', 'none'),
    'C':[1, 10,0.1,100],
    'solver':('newton-cg','lbfgs', 'liblinear', 'sag', 'saga'),
    
    }

GS = GridSearchCV(clf, parameters,n_jobs=5,scoring='f1')
#GS.fit(train_X, train_y['outcome'])


#sorted(GS.cv_results_.keys())

In [16]:
#print(GS.best_params_)
#print(pd.DataFrame(GS.cv_results_))

In [17]:
from sklearn.feature_selection import RFE

best_parameters = {
    'C': 1, 'penalty': 'l1', 'solver': 'saga'
}

clf = LogisticRegression(**best_parameters,class_weight={1:1,0:0.1},max_iter=10000) 

clf.fit(train_X, train_y['outcome'])
result = clf.predict(val_X)


print(pd.DataFrame({
    'train':tools.get_performance(train_y,clf.predict(train_X)),
    'val':tools.get_performance(val_y,result)
    }
    ))




                                        train                        val
confusion matrix   [[4894, 2286], [266, 452]]  [[3669, 1761], [61, 100]]
acc                                   0.67688                   0.674119
precision                            0.165084                   0.053735
f1_score                             0.261574                   0.098912
recall                               0.629526                   0.621118
matthews_corrcoef                     0.18795                   0.105332
